# Validating Simulation Grids via Maxwell's Equations

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from mu2e.dataframeprod import DataFrameMaker
#from mu2e.src.fiteval_c import FitFunctionMaker
import six.moves.cPickle as pkl
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,8)
#ffm= FitFunctionMaker("../mu2e/src/param_825.csv")
#def ffm_wrapper(x):
#    out = ffm.mag_field_function(x.X,x.Y,x.Z,True)
#    return pd.Series({'Bx':out[0], 'By':out[1], 'Bz':out[2]})
from mu2e import mu2e_ext_path

In [ ]:
#df_GA05= DataFrameMaker(mu2e_ext_path+'datafiles/FieldMapsGA05/DSMap',use_pickle = True).data_frame
# df_Mau10 = DataFrameMaker(mu2e_ext_path+'datafiles/Mau10/Standard_Maps/Mu2e_DSMap',input_type='pkl').data_frame
#df_Bus = DataFrameMaker(mu2e_ext_path+'datafiles/FieldMapsPure/TS5_DS_longbus',use_pickle = True).data_frame
#df_Bessel = pkl.load(open('df_Bessel.p','rb'))
df_Glass = DataFrameMaker(mu2e_ext_path+'datafiles/FieldMapsPure/test_helix_5L_detail', input_type='pkl').data_frame
# df_Pel = DataFrameMaker(mu2e_ext_path+'datafiles/synth/synth3',input_type='pkl').data_frame

In [ ]:
df_Glass.Z.unique()

In [ ]:
# df_Mau10_selection = df_Mau10.query('6000<Z<12000 and -800<X<800 and -100<Y<100')[['X','Y','Z','Bx','By','Bz']]
#df_GA05_selection = df_GA05.query('6000<Z<12000 and -800<X<800 and -100<Y<100')[['X','Y','Z','Bx','By','Bz']]
#df_Bessel_selection = df_Bessel.query('6000<Z<12000 and -600<X<600 and -600<Y<600')[['X','Y','Z','Bx','By','Bz']]
#df_Bus_selection = df_Bus.query('6000<Z<12000 and -800<X<800 and -100<Y<100')[['X','Y','Z','Bx','By','Bz']]
df_Glass_selection = df_Glass.query('-50<=Z<=50 and -1000<X<1000 and -1000<Y<1000')[['X','Y','Z','Bx','By','Bz']]
df_combo = df_Glass.query('-50<=Z<=50 and -1000<X<1000 and -1000<Y<1000')[['X','Y','Z','Bx','By','Bz']]
# df_Pel_selection = df_Pel.query('-3000<Z<3000 and -800<X<800 and -100<Y<100')[['X','Y','Z','Bx','By','Bz']]

We have generated a dataframe out of the fitting function as well, in order to determine the qualities of an object with known perfect adherence to Maxwell's equations.

In [ ]:
#df_combo = pd.merge(df_Pel_selection,df_Glass_selection,on=['X','Y','Z'],suffixes=('p','g'))
#df_combo = pd.merge(df_combo,df_Bessel_selection,on=['X','Y','Z'])
#df_combo = pd.merge(df_combo,df_Pel_selection,on=['X','Y','Z'])
df_combo.sort_values(['X','Y','Z'],inplace=True)


In [ ]:
df_combo.head()

For our first pass, we are going to calculate the divergence of the magnetic field.  We are going to do so by using using the Divergence Theorem,
$$\iiint_V(\nabla \cdot F)\ dV = \bigcirc \!\!\!\!\!\!\!\!\iint_S (F\cdot \mathbf{n})\ dS,$$
where we calculate the surface integral around some volume of the normal components of the magnetic field.  In our case, it makes logical sense to calculate the surface integral over a cubic surface.  Using the simulation grid, in which Bx,By,Bz triplets are given in 25mm spacings, we will create 3x3x3 cubes.  The central node on each face will represent the field vector for that face, and thus the total flux is the sum of all 6 faces (properly accounting for the direction of the normal).

We can accomplish this by creating 3D numpy arrays, and using vectorization in an intelligent manner.  We will label each cube by its central node.  We can then compare the total flux for each cube in the two simulations (Mau10 and GA05) that we are considering.  In principle, the flux should be 0 for all cubes; there are no magnetic monopoles (as far as we know).  In practice, the total flux will be non-zero, due to numeric errors.

In [ ]:
xs = df_combo.X.unique()
ys = df_combo.Y.unique()
zs = df_combo.Z.unique()

#bxs_Pel = df_combo.Bxp.reshape(len(xs),len(ys),len(zs))
#bys_Pel = df_combo.Byp.reshape(len(xs),len(ys),len(zs))
#bzs_Pel = df_combo.Bzp.reshape(len(xs),len(ys),len(zs))

bxs_Glass = df_combo.Bx.reshape(len(xs),len(ys),len(zs))
bys_Glass = df_combo.By.reshape(len(xs),len(ys),len(zs))
bzs_Glass = df_combo.Bz.reshape(len(xs),len(ys),len(zs))



In [ ]:
#div_Pel = bxs_Pel[:-2,1:-1,1:-1] - bxs_Pel[2:,1:-1,1:-1]\
#+ bys_Pel[1:-1,:-2,1:-1] - bys_Pel[1:-1,2:,1:-1]\
#+ bzs_Pel[1:-1,1:-1,:-2] - bzs_Pel[1:-1,1:-1,2:]

div_Glass = bxs_Glass[:-2,1:-1,1:-1] - bxs_Glass[2:,1:-1,1:-1]\
+ bys_Glass[1:-1,:-2,1:-1] - bys_Glass[1:-1,2:,1:-1]\
+ bzs_Glass[1:-1,1:-1,:-2] - bzs_Glass[1:-1,1:-1,2:]

In [ ]:
# curlxy_Pel = bxs_Pel[1:-1,:-2,1:-1] - bxs_Pel[1:-1,2:,1:-1]\
# + bys_Pel[:-2,1:-1,1:-1] - bys_Pel[2:,1:-1,1:-1]
# 
# curlxz_Pel = bxs_Pel[1:-1,1:-1,:-2] - bxs_Pel[1:-1,1:-1,2:]\
# + bzs_Pel[:-2,1:-1,1:-1] - bzs_Pel[2:,1:-1,1:-1]
# 
# curlyz_Pel = bys_Pel[1:-1,1:-1,:-2] - bys_Pel[1:-1,1:-1,2:]\
# + bzs_Pel[1:-1,:-2,1:-1] - bzs_Pel[1:-1,2:,1:-1]


curlxy_Glass = bxs_Glass[1:-1,:-2,1:-1] - bxs_Glass[1:-1,2:,1:-1]\
+ bys_Glass[:-2,1:-1,1:-1] - bys_Glass[2:,1:-1,1:-1]

curlxz_Glass = bxs_Glass[1:-1,1:-1,:-2] - bxs_Glass[1:-1,1:-1,2:]\
+ bzs_Glass[:-2,1:-1,1:-1] - bzs_Glass[2:,1:-1,1:-1]

curlyz_Glass = bys_Glass[1:-1,1:-1,:-2] - bys_Glass[1:-1,1:-1,2:]\
+ bzs_Glass[1:-1,:-2,1:-1] - bzs_Glass[1:-1,2:,1:-1]

In [ ]:
cube_centers = []
for x in xs[1:-1]:
    for y in ys[1:-1]:
        for z in zs[1:-1]:
            cube_centers.append('x{0}_y{1}_z{2}'.format(x,y,z))

In [ ]:
# df_maxwell = pd.DataFrame({'div_Pel':div_Pel.flatten(),'div_Glass':div_Glass.flatten(),
#                           'curlxy_Pel':curlxy_Pel.flatten(),'curlxy_Glass':curlxy_Glass.flatten(), 
#                           'curlxz_Pel':curlxz_Pel.flatten(),'curlxz_Glass':curlxz_Glass.flatten(), 
#                           'curlyz_Pel':curlyz_Pel.flatten(),'curlyz_Glass':curlyz_Glass.flatten(), 
#                           'b_tot_Pel':np.sqrt(bxs_Pel[1:-1, 1:-1, 1:-1]**2+bys_Pel[1:-1, 1:-1, 1:-1]**2+bzs_Pel[1:-1, 1:-1, 1:-1]**2).flatten(),
#                           'b_tot_Glass':np.sqrt(bxs_Glass[1:-1, 1:-1, 1:-1]**2+bys_Glass[1:-1, 1:-1, 1:-1]**2+bzs_Glass[1:-1, 1:-1, 1:-1]**2).flatten()},
#                           index=cube_centers)
df_maxwell = pd.DataFrame({'div_Glass':div_Glass.flatten(),
                          'curlxy_Glass':curlxy_Glass.flatten(), 
                          'curlxz_Glass':curlxz_Glass.flatten(), 
                          'curlyz_Glass':curlyz_Glass.flatten(), 
                          'b_tot_Glass':np.sqrt(bxs_Glass[1:-1, 1:-1, 1:-1]**2+bys_Glass[1:-1, 1:-1, 1:-1]**2+bzs_Glass[1:-1, 1:-1, 1:-1]**2).flatten()},
                          index=cube_centers)
df_maxwell.head()

In [ ]:
df_maxwell['div_Pel_rel'] = df_maxwell['div_Pel']/df_maxwell['b_tot_Pel']
df_maxwell['div_Glass_rel'] = df_maxwell['div_Glass']/df_maxwell['b_tot_Glass']

In [ ]:
df_maxwell['curlxy_Pel_rel'] = df_maxwell['curlxy_Pel']/df_maxwell['b_tot_Pel']
df_maxwell['curlxy_Glass_rel'] = df_maxwell['curlxy_Glass']/df_maxwell['b_tot_Glass']

In [ ]:
df_maxwell['curlxz_Pel_rel'] = df_maxwell['curlxz_Pel']/df_maxwell['b_tot_Pel']
df_maxwell['curlxz_Glass_rel'] = df_maxwell['curlxz_Glass']/df_maxwell['b_tot_Glass']

In [ ]:
df_maxwell['curlyz_Pel_rel'] = df_maxwell['curlyz_Pel']/df_maxwell['b_tot_Pel']
df_maxwell['curlyz_Glass_rel'] = df_maxwell['curlyz_Glass']/df_maxwell['b_tot_Glass']

In [ ]:
df_maxwell[['div_Glass']].plot.hist(alpha=0.5, bins=100, logy=True, title='Divergence')
# df_maxwell[['div_Pel_rel','div_Glass_rel']].plot.hist(alpha=0.5, bins=100, logy=True, title='Relative Divergence')

In [ ]:
df_maxwell[['curlxy_Glass']].plot.hist(alpha=0.5, bins=100, logy=True, title='XY Curl of Cubes')

In [ ]:
df_maxwell[['curlxz_Glass']].plot.hist(alpha=0.5, bins=100, logy=True, title='XZ Curl')

In [ ]:
df_maxwell[['curlyz_Glass']].plot.hist(alpha=0.5, bins=100, logy=True, title='YZ Curl')